# CF 1.6 NLP & Network Analysis Using 20th Century Key Events

## In this section the previously generated text doc and country list will be used to confirm the qty. of relationships/interactions between countries during 20th century events.
 - Step 1: Installing / Importing necessary libraries and modules
 - Step 2: Generate a list of entities (Key words, phrases etc.)
 - Step 3: Importing Country List and Filtering out entities with no country references
 - Step 4: Creating relationships between countries identified in the entities
 - Step 5: Aggregation
 - Step 6: Exporting

### Step 1: Library / module importing

In [4]:
#Due to Venv / Kernel issues, Spacy could not be installed into the environment prior to actioning the script
!pip install spacy

  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/39/e1/08681583569f435347ced0535b27c073fcc9a927d9b4293c963092f2d01c/spacy-3.7.5-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for spacy-legacy<3.1.0,>=3.0.11 from https://files.pythonhosted.org/packages/c3/55/12e842c70ff8828e34e543a2c7176dac4da006ca6901c9e8b43efab8bc6b/spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata
  Obtaining dependency information for spacy-loggers<2.0.0,>=1.0.0 from https://files.pythonhosted.org/packages/33/78/d1a1a026ef3af911159398c939b1509d5c36fe524c7b644f34a5146c4e16/spacy_loggers-1.0.5-py3-none-any.whl.metadata
  Obtaining dependency information for murmurhash<1.1.0,>=0.28.0 from https://files.pythonhosted.org/packages/71/46/af01a20ec368bd9cb49a1d2df15e3eca113bbf6952cc1f2a47f1c6801a7f/murmurhash-1.0.10-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for cymem<2.1.0,>=2.0.2 from https://files.pythonhosted.org/packages/c1/c3/dd

In [5]:
#Importing required libraries
import pandas as pd
import numpy as np
import spacy
from spacy import displacy
import networkx as nx
import os
import matplotlib.pyplot as plt
import scipy
import re

In [6]:
# Download English module

!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 330.3 kB/s eta 0:00:39
      --------------------------------------- 0.2/12.8 MB 2.5 MB/s eta 0:00:06
     ------- -------------------------------- 2.3/12.8 MB 18.3 MB/s eta 0:00:01
     ---------------- ----------------------- 5.3/12.8 MB 30.7 MB/s eta 0:00:01
     ------------------------- -------------- 8.2/12.8 MB 37.6 MB/s eta 0:00:01
     ---------------------------------- ---- 11.2/12.8 MB 65.6 MB/s eta 0:00:01
     --------------------------------------  12.8/12.8 MB 73.1 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 54.7 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [8]:
# Load spacy English module

NER = spacy.load("en_core_web_sm")

In [9]:
# Load the book

with open('key_events_of_the_20th_century.txt', 'r', errors='ignore') as file:
    data = file.read().replace('\n', '')

In [10]:
book = NER(data)

In [11]:
# Visualize identified entities

displacy.render(book[273:20000], style = "ent", jupyter = True)

### Step 2: Isolating Text Entities

In [12]:
df_sentences = [] # empty shell to store results

# Loop through sentences, get entity list for each sentence
for sent in book.sents:
    entity_list = [ent.text for ent in sent.ents]
    df_sentences.append({"sentence": sent, "entities": entity_list})
    
df_sentences = pd.DataFrame(df_sentences)

In [13]:
df_sentences.head(10)

sentence  \
0  (Key, events, of, the, 20th, century, -, Wikip...   
1                           (the, wars1.2.1Economic)   
2  (depression1.2.2The, rise, of, dictatorship1.3...   
3  (World, War, II, (, 1939â€“1945)1.3.1The, war,...   
4  (days1.3.7The, war, in, the, Pacific1.3.7.1Bac...   
5  (decolonization1.4.2The, Cold, War, (, 1947â€“...   
6  (The, World, Wars, sparked, tension, between, ...   
7  (These, advancements, have, played, a, signifi...   
8  (Events, in, the, 20th, century[edit]The, worl...   
9  (The, 1900s, saw, the, decade, herald, a, seri...   

                                            entities  
0  [the 20th century - WikipediaJump, Navigation\...  
1                                                 []  
2                                                 []  
3                                     [World War II]  
4  [Pacific1.3.7.1Background1.3.8Japanese, Holoca...  
5  [decolonization1.4.2The Cold War, 1947â€“1991)...  
6  [the Cold War, the Space Race, the World Wide ...  
7                          [the 21st century, today]  
8            [the 20th, Edwardian, the 20th century]  
9                            [The 1900s, the decade]

### Step 3: Importing Country List and filtering irrelevant entities

In [16]:
# Import countries list generated from prior ex.

country_df = pd.read_csv("countries.csv", index_col = 0)
country_df.reset_index(drop=True, inplace=True)
country_df.head()

Country
0  Afghanistan
1      Albania
2      Algeria
3      Andorra
4       Angola

In [19]:
# Function to filter out entities not of interest

def filter_entity(ent_list, country_df):
    return [ent for ent in ent_list 
            if ent in list(country_df['Country'])]

In [20]:
df_sentences['country_entities'] = df_sentences['entities'].apply(lambda x: filter_entity(x, country_df))

In [22]:
df_sentences['country_entities'].head(20)

0                              []
1                              []
2                              []
3                              []
4                              []
5                              []
6                              []
7                              []
8                              []
9                              []
10                             []
11                             []
12                             []
13                             []
14                             []
15                             []
16                             []
17        [France, Italy, Russia]
18    [Germany, Bulgaria, Russia]
19              [Germany, Russia]
Name: country_entities, dtype: object

In [23]:
# Filter out sentences that don't have any character entities

df_sentences_filtered = df_sentences[df_sentences['country_entities'].map(len) > 0]

In [24]:
df_sentences_filtered.tail(10)

sentence  \
847   (", Indian, Independence, Day, :, everything, ...   
853   (Retrieved, 2018, -, 12, -, 18.^, ", The, Phil...   
901   (The, Moldovans, :, Romania, ,, Russia, ,, and...   
910   (Now, ,, North, Korea, may, be, the, one, true...   
945   (", Selling, ", Operation, Passage, to, Freedo...   
968   (", Stuck, in, Endless, Preliminaries, :, Viet...   
1143  (", Anti, -, American, Behavior, in, the, Midd...   
1146  (The, Rise, of, China, and, India, :, A, New, ...   
1147               (Singapore, :, World, Scientific, .)   
1156     (", The, SARS, epidemic, in, Hong, Kong, ", .)   

                                               entities   country_entities  
847   [Indian Independence Day, Partition, India, Pa...  [India, Pakistan]  
853   [2018-12-18.^, Philippines, 1898â€“1946, US Ho...      [Philippines]  
901   [Moldovans, Romania, Russia, the Politics of C...  [Romania, Russia]  
910                                  [North Korea, one]      [North Korea]  
945   [Thomas Dooley, the Religious Overtones of Ear...          [Vietnam]  
968   [Vietnam, the Battle of the Paris Peace Table,...          [Vietnam]  
1143          [Anti-American, the Middle East, Lebanon]          [Lebanon]  
1146                    [China, India, New Asian Drama]     [China, India]  
1147                      [Singapore, World Scientific]        [Singapore]  
1156                                        [Hong Kong]        [Hong Kong]

### Step 4: Generating Relationships between country values in the remaining entities

In [36]:
# Defining relationships 

# window size = 5 : this defines how many sentences will be looked at simultaneously 
relationships = [] # create an empty list

# The purpose of this section of the for loop is to iterate through the main sentences DF and essentially slices the DF into sections which compile 5 rows together, 
# and then convert them into a single list which will then be appended to a DF later.
for i in range(df_sentences_filtered.index[-1]):
    end_i = min(i+5, df_sentences_filtered.index[-1])
    country_list = sum((df_sentences_filtered.loc[i: end_i].country_entities), [])
    
    # Remove duplicated characters that are next to each other - in the 'country_list' we just corrected, this script will remove any countries that appear side by side 
    # in their respective list - e.g USA,USA, Mexico, Japan would be converted to just USA, Mexico, Japan
    country_unique = [country_list[i] for i in range(len(country_list)) 
              if (i==0) or country_list[i] != country_list[i-1]]
    
    # Since some of the country_unique values might only contain 1 country, there would be no relationship so we only want to work with countries 
    # which have conenctions to others (hence the >1 filter). The script then iterates through the script to break down each list into 2 subsequent components. 
    # e.g list values: 'USA, Canada, Mexico, Japan' becomes a list of 'USA, Canada' - 'Canada, Mexico' - 'Mexico, Japan' - There are inherent issues where 
    # values that are linked by sentence, but not by immediate proximity are not fully accounted for.
    if len(country_unique) > 1:
        for idx, a in enumerate(country_unique[:-1]):
            b = country_unique[idx + 1]
            relationships.append({"source": a, "target": b})

In [49]:
print(end_i)

1156


In [37]:
relationship_df = pd.DataFrame(relationships)

In [39]:
relationship_df

source     target
0    France      Italy
1     Italy     Russia
2    France      Italy
3     Italy     Russia
4    Russia    Germany
..      ...        ...
707   India  Singapore
708   China      India
709   India  Singapore
710   China      India
711   India  Singapore

[712 rows x 2 columns]

In [41]:
# Sort the cases with a->b and b->a

relationship_df = pd.DataFrame(np.sort(relationship_df.values, axis = 1), columns = relationship_df.columns)
relationship_df.head(5)

source  target
0   France   Italy
1    Italy  Russia
2   France   Italy
3    Italy  Russia
4  Germany  Russia

### Step 5: Aggregation - Summarizing the qty of interactions between each grouping

In [42]:
# Summarize the interactions

relationship_df["value"] = 1
relationship_df = relationship_df.groupby(["source","target"], sort=False, as_index=False).sum()

In [43]:
relationship_df.head(10)

source   target  value
0    France    Italy     10
1     Italy   Russia      6
2   Germany   Russia     26
3  Bulgaria  Germany      6
4  Bulgaria   Russia      6
5   Germany    Italy     26
6   Austria  Germany     11
7   Germany    Spain      1
8    France   Poland     11
9    France  Germany     31

### Step 6: Exporting

In [44]:
relationship_df.to_csv('country_relationship.csv')